**import packages used**

---

---

# Mostrar o que queremos fazer de minimizar

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from pyXRD import pyXRDCodes
from scipy.signal import find_peaks
from scipy.optimize import minimize
from matplotlib.gridspec import GridSpec

In [33]:
# ========== OPTIMIZED CELL CLASS ==========
class Cell:
    def __init__(self, unit_params, unit_angles, lamb,bounds_params = [1,25]  , bounds_angles = [50,125] ):
        self.unit_params = np.array(unit_params, dtype=float)
        self.unit_angles = np.array(unit_angles, dtype=float)
        self.lamb = lamb

        self.bounds_params = bounds_params
        self.bounds_angles = bounds_angles

    
    def give_theta2(self):
        """Calculate 2θ angles for the current cell"""
        _, _, calculated_theta, _ = pyXRDCodes.generate_hkls(
            self.lamb, 10, self.unit_params, self.unit_angles
        )
       # Handle both array and empty cases properly
        if len(calculated_theta) > 0:
            return np.round( 2*np.array(calculated_theta), 2)
        return np.array([])  # Return empty array instead of list

    def mutate(self,  change_prob=0.5 , max_shift_param = 3.5 , max_shift_angle = 7):
        """Adaptive mutation"""
        for i in range(3):
            if random.random() < change_prob:
                value = self.unit_params[i] + np.random.uniform(-max_shift_param,max_shift_param+0.01)
                self.unit_params[i] = value
                if value > self.bounds_params[1]:
                    self.unit_params[i] = value - self.bounds_params[1] + self.bounds_params[0]
                if value < self.bounds_params[0]:
                    self.unit_params[i] = self.bounds_params[-1] - abs(value)

            if random.random() < change_prob:
                value = self.unit_angles[i]+ np.random.uniform(-max_shift_angle,max_shift_angle+0.01)
                self.unit_angles[i] = value
                if value > self.bounds_angles[1]:
                    self.unit_angles[i] = value - self.bounds_angles[1] + self.bounds_angles[0]
                if value < self.bounds_angles[0]:
                    self.unit_angles[i] = self.bounds_angles[1] - abs(value)

    def crossover(self, other, inherit_prob=0.7):
        """Blend two cells"""
        child = self.copy()

        for i in range(3):
            #if random.random() > inherit_prob:
            child.unit_params[i] = other.unit_params[i]*(1-inherit_prob) + self.unit_params[i]*(inherit_prob)
            #if random.random() > inherit_prob:
            child.unit_angles[i] = other.unit_angles[i]*(1-inherit_prob) + self.unit_angles[i]*(inherit_prob)#other.unit_angles[i]


        return child

    def copy(self):
        return Cell(self.unit_params.copy(), self.unit_angles.copy(), self.lamb,self.bounds_params  , self.bounds_angles)


In [34]:
def generate_initial_cell( param_bounds=[1,20],angles_bounds = [50,124]):
    # Filter params and angles within given bounds
    a = np.random.uniform(param_bounds[0],param_bounds[1])
    c = np.random.uniform(param_bounds[0],param_bounds[1])
    b = np.random.uniform(param_bounds[0],param_bounds[1])
    alpha = np.random.uniform(angles_bounds[0],angles_bounds[1])
    beta = np.random.uniform(angles_bounds[0],angles_bounds[1])
    gamma = np.random.uniform(angles_bounds[0],angles_bounds[1])
    
    
    return [a, b, c], [alpha, beta, gamma]

In [35]:
def Vol_calc(unit_params,unit_angles,V_max):
    a = pyXRDCodes.abcToxyz([1,0,0],unit_params,unit_angles)
    b = pyXRDCodes.abcToxyz([0,1,0],unit_params,unit_angles)
    c = pyXRDCodes.abcToxyz([0,0,1],unit_params,unit_angles)

    V = abs(np.dot(a,np.cross(b,c)))
    if V>=V_max:
        return False
    return True
    
Vol_calc([18,18,19],[110,110,120],3000)

False

In [36]:
def cosine_fitness(cell, theta2_exp, power=1, peak_tol=0.02):
    """
    Cosine similarity-based fitness with power scaling for peak matching
    
    Args:
        struct: Candidate structure
        theta2_exp: Experimental 2θ angles (degrees)
        int_exp: Experimental intensities
        lamb: Wavelength (Å)
        alltheta2: All possible theoretical 2θ angles
        power: Exponent for cosine similarity (higher = stricter)
        peak_tol: Peak matching tolerance (degrees)
        
    Returns:
        Fitness score (higher is better)
    """
    if not Vol_calc(cell.unit_params,cell.unit_angles,2000):
        return 1e3
    alltheta2 = np.arange(min(theta2_exp)-2 , max(theta2_exp)+2 , peak_tol)
    # Calculate theoretical pattern
    thetas2 = np.array(cell.give_theta2())

    mask = (thetas2>=min(alltheta2))&(thetas2<=max(alltheta2))
    thetas2 = thetas2[mask]


    # Create binary peak vectors (1=peak, 0=no peak)
    def make_binary_vector(peaks_theta):
        vec = np.zeros(len(alltheta2))
        for t in peaks_theta:
            idx = np.argmin(np.abs(alltheta2 - t))
            #if np.abs(alltheta2[idx] - t) <= peak_tol:
            vec[idx] = 1
        return vec
    
    # Experimental binary vector (only peaks above threshold)
    v = make_binary_vector(theta2_exp)
    u = make_binary_vector(thetas2)
    
    # Calculate cosine similarity
    dot_product = np.sum(u*v)#np.dot(u, v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    
    # Handle edge cases
    if norm_u == 0 or norm_v == 0:
        return 1e3
    
    cosine_sim = dot_product / (norm_u * norm_v)
    
    # Apply power scaling
    fitness = (cosine_sim)**power
    
    return -fitness #- atom_penalty  # Higher is better



Fazer GA

In [43]:
def run_ga(observed_peaks,power=1, lamb=1.5406, generations=10, numb_pop=60,peak_tol=0.02,
           max_shift_param = 2.5,max_shift_angles = 5,
           param_bounds = [1,25], angle_bounds = [5,125], change_prob=0.1, elit_frac=0.2, early_stag=3, inherit_prob=0.7):

    # Initialize population
    population = []
    for _ in range(numb_pop):
        abc, angles = generate_initial_cell( param_bounds, angle_bounds)
        population.append(Cell(abc, angles, lamb,param_bounds,angle_bounds))
    
    best_cell = None
    best_fitness = float('inf')
    no_improvement = 0
    stagnation_count = 0
    Cells_history = [population]
    Fitness_historu = []
    best_history = []

    for gen in range(generations):
        # Evaluate fitness
        fitness = [cosine_fitness(s,observed_peaks,power=1,peak_tol=peak_tol) for s in population]
        Fitness_historu.append(fitness)
        current_best = min(fitness)


        # Update best solution
        if current_best < best_fitness:
            best_fitness = current_best
            #fitness_history.append(best_fitness)
            best_cell = population[np.argmin(fitness)]
            #best_history.append(best_cell)
            no_improvement = 0
            stagnation_count = 0
        else:
            no_improvement += 1
            stagnation_count += 1
        

        # ========== Anti-stagnation measures ==========
        if stagnation_count >= early_stag:
            print(f"\n Stagnation detected at generation {gen+1}.\n Applying corrective measures...")

            # Rebuild population with diversity
            new_population = []

            # Do not Keep best cell (elitism)
            new_population.append(best_cell.copy())
            # Clone best with mutation (local exploration)
            for _ in range(numb_pop-1):
                abc, angles = generate_initial_cell( param_bounds, angle_bounds)
                new_population.append(Cell(abc, angles, lamb,param_bounds,angle_bounds))
                
            # Reassign population
            population = new_population
            
            fitness = [cosine_fitness(s,observed_peaks,power=1,peak_tol=peak_tol) for s in population]
            current_best = min(fitness)
            no_improvement = 0
            # Update best solution
            if current_best < best_fitness:
                best_fitness = current_best
                #fitness_history.append(best_fitness)
                best_cell = population[np.argmin(fitness)]
                #best_history.append(best_cell)
                no_improvement = 0
                stagnation_count = 0
            else:
                no_improvement += 1
                stagnation_count += 1
            print("Applied anti-stagnation measures: mutation boost + new individuals + hybridization")
    
        #================================================================
        best_history.append(best_cell)

        # ========== Selection ==========
        parents = []
        for _ in range(numb_pop):
            candidates = random.sample(range(numb_pop), 3)
            winner = candidates[np.argmin([fitness[c] for c in candidates])] if random.random() < 0.8 else random.choice(candidates)
            parents.append(population[winner])
        # ========== New generation ==========
        new_pop = []
        elite_size = int(elit_frac * numb_pop)
        elites = sorted(zip(population, fitness), key=lambda x: x[1])[:elite_size]
        new_pop.extend([x[0] for x in elites])

        if best_cell and best_cell not in new_pop:
            new_pop[0] = best_cell

        while len(new_pop) < numb_pop:
            parent1, parent2 = np.random.choice(population, 2)
            child = parent1.crossover(parent2, inherit_prob)
            current_change_prob = min(0.5, change_prob * (1 + stagnation_count / early_stag))
            child.mutate(change_prob=current_change_prob,max_shift_param=max_shift_param,max_shift_angle=max_shift_angles)
            new_pop.append(child)

        population = new_pop

        # ========== Reporting ==========
        print(f"Gen {gen+1}: Best Fitness = {best_fitness:.4f} \n"
              f"Params = {best_cell.unit_params if best_cell else None} | "
              f"Angles = {best_cell.unit_angles if best_cell else None}")
        #unique_params = len({tuple(cell.unit_params) for cell in population})
        #print(f"Diversity: {unique_params}/{numb_pop} unique parameter sets")
        Cells_history.append(population)
        
#=====================================================================
    # Evaluate fitness
    fitness = [cosine_fitness(s,observed_peaks,power=1,peak_tol=peak_tol) for s in population]
    current_best = min(fitness)
    Fitness_historu.append(fitness)
    
        # Update best solution
    if current_best < best_fitness:
        best_fitness = current_best
        #fitness_history.append(best_fitness)
        best_cell = population[np.argmin(fitness)]
            #best_history.append(best_cell)
        no_improvement = 0
        stagnation_count = 0
    else:
        no_improvement += 1
        stagnation_count += 1
    
    best_history.append(best_cell)


    return best_cell, best_history,Cells_history,Fitness_historu


In [44]:
# Your target structure (Ti-C-Al-O)
lamb = 1.5406
target_Atoms = ['Ti', 'C', 'O']

structures=['cubic','hexagonal','tetragonal','orthorhombic','monoclinic','trigonal','triclinic']
params_grid = np.arange(1,25.5,0.01)
angles_grid = np.arange(50,125,1)
target_structure = ['cubic','hexagonal','tetragonal','orthorhombic','monoclinic','trigonal','triclinic']

giving_params = [[5,5,5],[3,3,15],[4,4,20],[5.2,7.8,15],[8,4,6],[7,7,7],[5,12,8]]
giving_angles = [[90,90,90],[90,90,120],[90,90,90],[90,90,90],[90,115,90],[70,70,70],[65,80,110]]



    #params = generate_initial_cell(target_structure[i], params_list = np.arange(1,30,0.1), angles_list = angles_grid,
    #                      param_bounds=[1,30],unit_bounds = [50,124]) 
targe_unit_params = np.array(giving_params[1]) 
targe_unit_angles = np.array(giving_angles[1]) 
target_Positions = [
                [pyXRDCodes.abcToxyz([1/2, 1/3, 1/3], targe_unit_params, targe_unit_angles)],
                [pyXRDCodes.abcToxyz([0, 0, 0], targe_unit_params, targe_unit_angles),
                pyXRDCodes.abcToxyz([1/2, 1/2, 1/2], targe_unit_params, targe_unit_angles)],
                [pyXRDCodes.abcToxyz([0, 1/2, 1/2], targe_unit_params, targe_unit_angles)]
    ]
# Run GA
# Generate expected peaks
data = pyXRDCodes.simulate_xrd(lamb, 10, targe_unit_params, targe_unit_angles, target_Positions, target_Atoms)
observed_theta = np.round(np.array(data['two_thetas']), 2)
U = data['intensities']

best_cell, Best_Hist,All_Hist,Fitness_Hist = run_ga(
                observed_theta,
                peak_tol=0.02,
                power=3,
                lamb=lamb,
                generations=90,
                numb_pop= 120,
                change_prob=0.8,
                inherit_prob=0.7,
                elit_frac=0.2,
                early_stag=5,
                param_bounds=[1,25],
                angle_bounds=[50,125],
                max_shift_angles=20,
                max_shift_param=9
)



Gen 1: Best Fitness = -0.0733 
Params = [21.42433283  6.05484318  3.47969837] | Angles = [105.20996566  54.30305501 119.33661301]
Gen 2: Best Fitness = -0.0832 
Params = [ 6.48111053 21.5766118  18.41045772] | Angles = [123.60355592  68.34811409  82.24682167]
Gen 3: Best Fitness = -0.0832 
Params = [ 6.48111053 21.5766118  18.41045772] | Angles = [123.60355592  68.34811409  82.24682167]
Gen 4: Best Fitness = -0.0832 
Params = [ 6.48111053 21.5766118  18.41045772] | Angles = [123.60355592  68.34811409  82.24682167]
Gen 5: Best Fitness = -0.0832 
Params = [ 6.48111053 21.5766118  18.41045772] | Angles = [123.60355592  68.34811409  82.24682167]
Gen 6: Best Fitness = -0.0832 
Params = [ 6.48111053 21.5766118  18.41045772] | Angles = [123.60355592  68.34811409  82.24682167]

 Stagnation detected at generation 7.
 Applying corrective measures...
Applied anti-stagnation measures: mutation boost + new individuals + hybridization
Gen 7: Best Fitness = -0.0832 
Params = [ 6.48111053 21.5766118  